
<div>
<img src="https://clms-prod.eea.europa.eu/static/media/EU-Cop-logo.4120d6e3.svg" width="200"/>
</div>
<div>
<img src="https://clms-prod.eea.europa.eu/static/media/ccl-icon-land-text.bfaba89b.svg" width="200"/>
</div>

***
The goal of this notebook is to help you kickstart your project and to provide an example on how to download a dataset. 
You can see all the API documentation here: <br> https://eea.github.io/clms-api-docs/ <br>
You can view information regarding the Machine2Machine download here: <br>https://clms-prod.eea.europa.eu/en/how-to-guides/download-spatial-data/m2m-download
***

## Sign up and token declaration

* Go to https://clms-prod.eea.europa.eu/en  
* Register using EU login
* After signing in to the portal navigate to 'My profile' under your username
* Create a new token, press Copy to clipboard and paste it in a .json file called 'my_saved_key.json' in the same folder as this notebook



# Prepare the environment

To run this notebook you need to install the following dependencies.

```bash
pip install pyjwt requests cryptography

```

After installing them, you are ready to proceed

In [1]:
import json
import jwt
import time
import requests

base_url = 'http://clms.127.0.0.1.nip.io'

## Do the authentication steps

First load the service key downloaded from the website and create a grant request

In [2]:
# Load saved key from filesystem
service_key = json.load(open('my_saved_key.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

With the grant, you can request the access token to use the API

In [3]:
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

07oMdfuv2ULcbmcHkHeY0x-bNlHNibu-xK1yEeErCoRdwwMCKMS5IGE0w_bXaPA1ZKPKzxxqPQpkSAPSBKLX-Q==


### Find the items that can be downloaded

This step makes a search query and prints the title, UID and link of the first 25 items found.



## Search prepackaged files that can be downloaded

You can use the `@search` endpoint to search for datasets

In [ ]:
url_downloadable_prepackaged = f"{base_url}/api/@search?portal_type=DataSet&metadata_fields=UID&metadata_fields=downloadable_files"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_downloadable_prepackaged = requests.get(url_downloadable_prepackaged, headers=headers)

json_downloadable_prepackaged = response_downloadable_prepackaged.json()

for item in json_downloadable_prepackaged.get('items', []):
    print('Product information:')
    print('\n')
    print(3*' '+'Product title: "'+ item['title'].replace(':','')+'"')
    print(3*' '+'UID: "'+item['UID'].replace(' ','')+'"')
    print(3*' '+'Product link: '+ item['@id']+'')
    print('\n')
    
    for downloadable_file in item.get('downloadable_files', {}).get('items', []):
        print('Download option for "'+item['title'].replace(':','')+'":')
        display(downloadable_file)
        print(2*'\n')

    print('\n')
    print(100*'_') 

### Download example dataset

You are welcome to specify the configuration of your data request by using the `data` variable. 
If the request is successful, then a task ID will be created under your account and when the data extraction is ready to be downloaded you will receive an email with the download link. 


Further documentation can be found here: https://eea.github.io/clms-api-docs/download.html#request-the-download

In [ ]:
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'a8d945f0edd143a0a5240c28bafa23da', 'DatasetDownloadInformationID': 'f328dbcc-9069-4240-b8bb-6d5df918671a', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITC11'}, {'DatasetID': 'a8d945f0edd143a0a5240c28bafa23da', 'DatasetDownloadInformationID': 'f328dbcc-9069-4240-b8bb-6d5df918671a', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITC1'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

***

### Download example prepackaged file

You are welcome to specify the configuration of your data request by using the json variable. 
If the request is successful, then a task ID will be created under your account and when the data extraction is ready to be downloaded you will receive an email with the download link. <br> <br>
Further documentation can be found here: https://eea.github.io/clms-api-docs/download.html#download-prepackaged-files

In [ ]:
url_download_prepackaged = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [
    {
     'DatasetID': 'd30959acacf84e418c658ecaf6673ebf', 
     'FileID': '7b2c4ca3-749c-4c3d-b4cd-a848e7465175'
    }, 
    {
     'DatasetID': 'd30959acacf84e418c658ecaf6673ebf', 
     'FileID': '345b7d52-fa71-43da-a182-dca7a74a16e2'
    }
]}
print(url_download_prepackaged)
response_download_prepackaged = requests.post(url_download_prepackaged, headers=headers, json=data)
print('Response:',response_download_prepackaged)
print('Message with TaskID:\n'+response_download_prepackaged.text)